Script para criação de modelo de classificação de tipos celulares em imagens de lâminas de sangue humano.
Modelo baseado no algoritmo de Rede Neural Convolucional, treinado com o dataset [Blood Cell Images do Paul Mooney no Kaggle](https://www.kaggle.com/paultimothymooney/blood-cells)

In [ ]:
#Carregando o kaggle para o Google Colab
#Adicionando o token pessoal para autenticação e utilização do kaggle API

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"","key":""}

#Importações

import json
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
from keras.preprocessing import image
from keras.models import load_model
import os
import tensorflow as tf
import numpy
import os
from PIL import Image

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

Using TensorFlow backend.


In [ ]:
#Baixando e descompactando o dataset

!kaggle datasets download -d paultimothymooney/blood-cells

 93% 101M/108M [00:01<00:00, 81.3MB/s] 
100% 108M/108M [00:01<00:00, 86.1MB/s]


In [ ]:
!unzip 'blood-cells.zip'

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8167.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8283.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8324.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8379.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8577.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8587.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8655.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8701.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_872.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8864.jpeg  
  inflating: dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE/_9_8931.jpeg  
  inflating: datas

In [ ]:
# Criando um generator para preprocessamento das imagens.

generator = image.ImageDataGenerator(
    rescale = 1./255, #pôr escala de cor de 0 a 1
    featurewise_center  = False, #não usar média de 0 para os dados
    samplewise_center = True, #aplicar média 0 para cada amostra
    featurewise_std_normalization = False, #não normalizar os inputs por desvio padrão do conjunto de dados
    samplewise_std_normalization = True, #dividir cada amostra por desvio padrão
    zca_whitening = False, #não aplicar o branqueamento de imagens
    rotation_range = 30, #rotação aleatória das imagens em 10 graus
    width_shift_range=0.3, #deslocar a imagem no comprimento
    height_shift_range = 0.3,  #deslocar a imagem na altura
    horizontal_flip = True, #espelhar a imagem horizontalmente
    vertical_flip = True # espelhar a imagem verticalmente
)

In [ ]:
#Criar um dataset com as imagens no diretório de treino
#A pasta contém subpastas para cada uma das 4 classes preditas:
#Linfócitos
#Monócitos
#Eosinófilos
#Neutrófilos

dataset = generator.flow_from_directory(
    shuffle = True, #Misturar as amostras aleatoriamente
    batch_size = 32, #Configurando 32 amostras por batch
    target_size = (80, 80), #Reduzir o tamanho das imagens para 80x80
    directory = 'dataset2-master/dataset2-master/images/TRAIN'
)

Found 9957 images belonging to 4 classes.


In [ ]:
# Configurando a rede neural sequencial do modelo

def model():
    model = Sequential()
    #Setando a primeria camada convolucional 2d: convolução espacial de imagens, com ativação relu padrão
    model.add(Conv2D(64, (3,3), strides = (1, 1), activation = 'relu', input_shape = (80, 80, 3)))
    #Setando uma segunda camada conv2d
    model.add(Conv2D(80, (3,3), strides = (1, 1), activation = 'relu'))
    #Adicionando uma camada de maxpooling, escolhendo o maior valor numa janela de dimensão (2,2)
    model.add(MaxPool2D(pool_size = (2,2)))
    #Adicionando outra camada conv2d
    model.add(Conv2D(64, (3,3), strides = (1,1), activation = 'relu'))
    #Adicionando uma camada dropout dos inputs a uma taxa de 0.25
    model.add(Dropout(0.25))
    #Adicionando camada de achatamento, para transformar matriz em um array
    model.add(Flatten())
    #Adicionando uma camada densa
    model.add(Dense(128, activation = 'relu'))
    #Adicionando outra camada de dropout
    model.add(Dropout(0.5))
    #Finalizando com uma camada densa como output
    model.add(Dense(4, activation = 'softmax'))

    #Setando a loss function como categorical crossentropy para classificação categórica
    #adadelta como optimizer
    #E categorical accuracy como métrica de performance
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['categorical_accuracy'])
    
    return model

nn = model()

In [ ]:
#Encontrando o GPU disponível

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Treinando o modelo por 30 épocas utilizando o gpu

with tf.device("gpu:0"):
   nn.fit_generator(dataset, steps_per_epoch = None, epochs = 30, verbose = 1)

Epoch 1/30
312/312 [==============================] - 32s 103ms/step - loss: 1.3175 - categorical_accuracy: 0.3544
Epoch 2/30
312/312 [==============================] - 32s 103ms/step - loss: 0.9270 - categorical_accuracy: 0.6092
Epoch 3/30
312/312 [==============================] - 32s 103ms/step - loss: 0.6062 - categorical_accuracy: 0.7421
Epoch 4/30
312/312 [==============================] - 32s 103ms/step - loss: 0.4975 - categorical_accuracy: 0.7896
Epoch 5/30
312/312 [==============================] - 32s 102ms/step - loss: 0.4440 - categorical_accuracy: 0.8159
Epoch 6/30
312/312 [==============================] - 32s 103ms/step - loss: 0.3974 - categorical_accuracy: 0.8412
Epoch 7/30
312/312 [==============================] - 32s 103ms/step - loss: 0.3788 - categorical_accuracy: 0.8481
Epoch 8/30
312/312 [==============================] - 32s 103ms/step - loss: 0.3452 - categorical_accuracy: 0.8597
Epoch 9/30
312/312 [==============================] - 32s 103ms/step - loss: 0.3

In [ ]:
#Salvando o modelo em arquivo.

nn.save('Model.h5')

In [ ]:
#Checando o tamanho do modelo antes de salvar.

print(os.stat('Model.h5'))
os.stat('Model.h5').st_size

os.stat_result(st_mode=33188, st_ino=2515729, st_dev=50, st_nlink=1, st_uid=0, st_gid=0, st_size=128574952, st_atime=1596419763, st_mtime=1596419763, st_ctime=1596419763)


128574952

In [ ]:
#model = load_model('Model.h5')

#Validando o modelo

correct = 0
wrong = 0
total = 0
dir = 'dataset2-master/dataset2-master/images/TEST/'
with tf.device("gpu:0"):
  for celltype in os.listdir(dir):
    for file in os.listdir(dir + celltype + '/'):
      image = Image.open(dir + celltype + '/' + file)
      image = image.resize((80, 80))
      image = numpy.array(image, dtype = 'float32')
      image/=255
      image = image.reshape(1, 80, 80, 3)
      prediction = nn.predict(image)
    #print(numpy.argmax(prediction))
      if numpy.argmax(prediction) == 0: correct+=1
      else: wrong+=1
      total+=1
    print(celltype + ' Result : ', 'Correct prediction %: ', (correct/total)*100, 'Wrong prediction : %', (wrong/total)*100)

EOSINOPHIL Result :  Correct prediction %:  14.606741573033707 Wrong prediction : % 85.39325842696628
LYMPHOCYTE Result :  Correct prediction %:  7.320997586484312 Wrong prediction : % 92.67900241351569
NEUTROPHIL Result :  Correct prediction %:  5.034815211569363 Wrong prediction : % 94.96518478843063
MONOCYTE Result :  Correct prediction %:  3.7796542018496178 Wrong prediction : % 96.22034579815039
